In [1]:
import pickle
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

/tmp/ipykernel_3594378/666084668.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
shap_tfbs = ["AHL20", "AT3G46070", "TRP5", "RAMOSA1", "ERF069", "REF6", "DOF1.5", "CDF5", "ARF2", "CDF2", "DOF5.3", "DOF5.7", "ARR1", "DOF3.2", "ABI3", "ARF16", "MNB1A", "DREB1D", "NID1", "MYB24"]

### Load in the motifs 

In [14]:
import Bio.SeqIO as SeqIO
import numpy as np 
from sklearn.preprocessing import LabelEncoder
LTRs = [rec for rec in SeqIO.parse("/data/xhorvat9/LTR_classification_data/Sequence_files/train_LTRs.fasta", "fasta") if rec.description.split()[3] != "NAN"]
LTR_motifs = pd.read_csv("/data/xhorvat9/LTR_classification_data/TFBS/LTR_train_motifCounts.csv", sep="\t").set_index("ID")


# LTR ordering is identical to its motif representation
LTR_sequence_df = pd.DataFrame({"sequence": [str(rec.seq) for rec in LTRs], "ID": [rec.id for rec in LTRs], "label": [rec.description.split()[3] for rec in LTRs]})
LTR_sequence_df.set_index("ID", inplace=True)
LTR_sequence_df = LTR_sequence_df.merge(LTR_motifs, on="ID")[["sequence", "label"]]
X_motifs = LTR_sequence_df.merge(LTR_motifs, on="ID").drop(["sequence", "label"], axis= 1)
print("Indices for LTR sequences and motifs are identical: ", all(X_motifs.index == LTR_sequence_df.index))


X = np.array(LTR_sequence_df["sequence"].tolist())

le = LabelEncoder()
y = le.fit_transform(LTR_sequence_df["label"])

#X_motifs = LTR_motifs

Indices for LTR sequences and motifs are identical:  True


In [4]:
tfidf = TfidfTransformer()
X = tfidf.fit_transform(X_motifs).toarray()

# score = clf.score(X, y)

### Load the trained GBC classifier 

In [12]:
X_motifs.shape

(141533, 654)

In [ ]:
import tqdm
tfbs_performance = {}
GBC = pickle.load(open("GBC_model_FULL.pkl", "rb"))
for tfbs in tqdm.tqdm(shap_tfbs):

    # Turn feature of interest to 0
    X_modified = X_motifs.copy()
    X_modified[[tfbs]] = 0

    # Scale using TF-IDF
    tfidf = TfidfTransformer()
    X = tfidf.fit_transform(X_modified).toarray()

    # Run the classifier
    modified_score = GBC.score(X, y)

    tfbs_performance[tfbs] = modified_score

  0%|          | 0/20 [00:00<?, ?it/s]

 25%|██▌       | 5/20 [00:50<02:31, 10.12s/it]

In [6]:
tfbs_performance

{'AHL20': 0.7645143035328629,
 'AT3G46070': 0.781191171884925,
 'TRP5': 0.775890145171908,
 'RAMOSA1': 0.7802210493492094,
 'ERF069': 0.780683012461455,
 'REF6': 0.7798283807038008,
 'DOF1.5': 0.7795281046808412,
 'CDF5': 0.7800824604155359,
 'ARF2': 0.7809717394066084,
 'CDF2': 0.7824615704436001,
 'DOF5.3': 0.7767101296961437,
 'DOF5.7': 0.7805790707611997,
 'ARR1': 0.7804058345941076,
 'DOF3.2': 0.781861018397681,
 'ABI3': 0.7790430434129835,
 'ARF16': 0.7812489172739556,
 'MNB1A': 0.7813066626629863,
 'DREB1D': 0.7801286567267604,
 'NID1': 0.7818148220864564,
 'MYB24': 0.78093709217319}

In [20]:
import tqdm
import random
randomized_performance = {}
GBC = pickle.load(open("GBC_model_FULL.pkl", "rb"))
for tfbs in tqdm.tqdm(shap_tfbs):

    # Turn feature of interest to 0
    X_modified = X_motifs.copy()
    X_modified[[tfbs]] = random.randint(0, max(X_modified[[tfbs]].values)[0])

    # Scale using TF-IDF
    tfidf = TfidfTransformer()
    X = tfidf.fit_transform(X_modified).toarray()

    # Run the classifier
    modified_score = GBC.score(X, y)

    randomized_performance[tfbs] = modified_score

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [02:05<00:00,  6.27s/it]


In [21]:
randomized_performance

{'AHL20': 0.604651968540312,
 'AT3G46070': 0.7806252670724243,
 'TRP5': 0.7389330961922691,
 'RAMOSA1': 0.7707854527815954,
 'ERF069': 0.7186529155646921,
 'REF6': 0.780683012461455,
 'DOF1.5': 0.7768602677076235,
 'CDF5': 0.7740076454895076,
 'ARF2': 0.778835160012473,
 'CDF2': 0.6825966946539319,
 'DOF5.3': 0.6528693683809348,
 'DOF5.7': 0.7128552785060113,
 'ARR1': 0.7799900677930868,
 'DOF3.2': 0.7804404818275261,
 'ABI3': 0.7623661750609214,
 'ARF16': 0.6954161710187442,
 'MNB1A': 0.7708547472484322,
 'DREB1D': 0.7786272766119625,
 'NID1': 0.7473523739129431,
 'MYB24': 0.7699654682573597}

## Do the same for insignificant features